In [1]:
# Provides ways to work with large multidimensional arrays
import numpy as np 
# Allows for further data manipulation and analysis
import pandas as pd
from pandas_datareader import data as web # Reads stock data 
import matplotlib.pyplot as plt # Plotting
import matplotlib.dates as mdates # Styling dates
%matplotlib inline

import datetime as dt # For defining dates
import mplfinance as mpf # Matplotlib finance

import time

# Used to get data from a directory
import os
from os import listdir
from os.path import isfile, join

#### Dates & Other Constants

In [2]:
# Define path to files
PATH = "../data/stock-list/"

# Start date defaults
ST_YEAR = 2017
ST_MONTH = 1
ST_DAY = 3
ST_DATE_STR = f"{ST_YEAR}-{ST_MONTH}-{ST_DAY}"
ST_DATE_DATETIME = dt.datetime(ST_YEAR, ST_MONTH, ST_DAY)

# End date defaults
EN_YEAR = 2021
EN_MONTH = 8
EN_DAY = 19
EN_DATE_STR = f"{EN_YEAR}-{EN_MONTH}-{EN_DAY}"
EN_DATE_DATETIME = dt.datetime(EN_YEAR, EN_MONTH, EN_DAY)

#### Get Stock File Names in a List

In [3]:
files = [x for x in listdir(PATH) if isfile(join(PATH, x))]
tickers = [os.path.splitext(x)[0] for x in files]
tickers

['A',
 'AA',
 'AAL',
 'AAME',
 'AAOI',
 'AAON',
 'AAP',
 'AAPL',
 'AAT',
 'AAWW',
 'ABBV',
 'ABC',
 'ABCB',
 'ABEO',
 'ABG',
 'ABIO',
 'ABM',
 'ABMD',
 'ABR',
 'ABT',
 'ABTX',
 'AC',
 'ACA',
 'ACAD',
 'ACBI',
 'ACC',
 'ACCO',
 'ACER',
 'ACGL',
 'ACHC',
 'ACHV',
 'ACIW',
 'ACLS',
 'ACM',
 'ACMR',
 'ACN',
 'ACNB',
 'ACOR',
 'ACRE',
 'ACRS',
 'ACRX',
 'ACTG',
 'ACU',
 'ACY',
 'ADBE',
 'ADC',
 'ADES',
 'ADI',
 'ADM',
 'ADMA',
 'ADMP',
 'ADNT',
 'ADP',
 'ADS',
 'ADSK',
 'ADT',
 'ADTN',
 'ADUS',
 'ADVM',
 'ADXS',
 'AE',
 'AEE',
 'AEHR',
 'AEIS',
 'AEL',
 'AEMD',
 'AEO',
 'AEP',
 'AERI',
 'AES',
 'AEY',
 'AFG',
 'AFI',
 'AFL',
 'AGCO',
 'AGE',
 'AGEN',
 'AGFS',
 'AGIO',
 'AGLE',
 'AGM',
 'AGNC',
 'AGO',
 'AGR',
 'AGRX',
 'AGS',
 'AGTC',
 'AGX',
 'AGYS',
 'AHH',
 'AHT',
 'AIG',
 'AIMC',
 'AIN',
 'AINC',
 'AIR',
 'AIRG',
 'AIRI',
 'AIRT',
 'AIT',
 'AIV',
 'AIZ',
 'AJG',
 'AJRD',
 'AJX',
 'AKAM',
 'AKBA',
 'AKR',
 'AKTS',
 'AL',
 'ALB',
 'ALBO',
 'ALCO',
 'ALDX',
 'ALE',
 'ALEC',
 'ALEX',
 'ALG'

#### Function that Returns a Dataframe from a CSV

In [4]:
def get_df_from_csv(ticker):
    try:
        df = pd.read_csv(PATH + ticker + '.csv')
    except FileNotFoundError:
        pass
        # print("File Doesn't Exist")
    else:
        return df

#### Function that Saves Dataframe to CSV

In [5]:
def save_dataframe_to_csv(df, ticker):
    df.to_csv(PATH + ticker + '.csv')

#### Return 1st Valid Date in Dataframe

In [6]:
def get_valid_dates(df, sdate, edate):  
    try:
        mask = (df['Date'] > sdate) & (df['Date'] <= edate) 
        sm_df = df.loc[mask]
        sm_df = sm_df.set_index(['Date'])
        sm_date = sm_df.index.min()
        last_date = sm_df.index.max()
        date_leading = '-'.join(('0' if len(x)<2 else '')+x for x in sm_date.split('-'))
        date_ending = '-'.join(('0' if len(x)<2 else '')+x for x in last_date.split('-'))
    except Exception:
        print("Date Corrupted")
    else:
        return date_leading, date_ending

#### Returns Return on Investment over Time

In [7]:
def roi_between_dates(df, sdate, edate):
    try: 
        start_val = df.loc[sdate,'Adj Close'] 
        end_val = df.loc[edate,'Adj Close']
        roi = ((end_val - start_val) / start_val)
    except Exception:
        print("Data Corrupted")
    else:
        return roi

#### Get Mean Between Dates

In [8]:
def get_mean_between_dates(df, sdate, edate):
    mask = (df['Date'] > sdate) & (df['Date'] <= edate)
    return df.loc[mask]["Adj Close"].mean()

#### Get Standard Deviation Between Dates

In [9]:
def get_sd_between_dates(df, sdate, edate):
    mask = (df['Date'] > sdate) & (df['Date'] <= edate)
    return df.loc[mask]["Adj Close"].std()

#### Get Coefficient of Variation Between Dates

In [10]:
def get_cov_between_dates(df, sdate, edate):
    mean = get_mean_between_dates(df, sdate, edate)
    sd = get_sd_between_dates(df, sdate, edate)
    return sd / mean

#### Merge Multiple Stocks in One Dataframe by Column Name

In [11]:
def merge_df_by_column_name(col_name, sdate, edate, *tickers):
    # Will hold data for all dataframes with the same column name
    mult_df = pd.DataFrame()
    
    for x in tickers:
        df = get_df_from_csv(x)
        df['Date'] = pd.to_datetime(df['Date'])
        # Use a mask to grab data between defined dates
        mask = (df['Date'] >= sdate) & (df['Date'] <= edate)
        mult_df[x] = df.loc[mask][col_name]
        
    return mult_df

---
NEW


#### 11 Stock Market Sectors

**Technology**: Manufacturing of electronics, software, or related to information technology

**Health Care**: Biotech, hospitals, medical devices, drugs

**Financials**: Banks, investment funds, and insurance

**Real Estate**: Residential, industrial, and retail real estate

**Energy**: Production and supply of energy

**Materials**: Mining, refining, chemical, and forestry

**Consumer Discretionary**: Retailers, apparel, media, durables and services

**Industrials**: Construction, machinery, fabrication, manufacturing, defense, and aerospace

**Utilities**: Direct providers of electric, gas, and water

**Consumer Staples**: Food, beverage as well as products consumers deem essential

**Telecommunication**: Cable, internet providers, wireless, and satellite

#### Get Sector Stocks

In [12]:
# For Windows
sec_df = pd.read_csv('../data/stock_sectors.csv')

# Get Industrials DF
indus_df = sec_df.loc[sec_df['Sector'] == "Industrials"]
health_df = sec_df.loc[sec_df['Sector'] == "Health Care"]
it_df = sec_df.loc[sec_df['Sector'] == "Information Technology"]
comm_df = sec_df.loc[sec_df['Sector'] == "Communication Services"]
staple_df = sec_df.loc[sec_df['Sector'] == "Consumer Staples"]
discretion_df = sec_df.loc[sec_df['Sector'] == "Consumer Discretionary"]
utility_df = sec_df.loc[sec_df['Sector'] == "Utilities"]
financial_df = sec_df.loc[sec_df['Sector'] == "Financials"]
material_df = sec_df.loc[sec_df['Sector'] == "Materials"]
restate_df = sec_df.loc[sec_df['Sector'] == "Real Estate"]
energy_df = sec_df.loc[sec_df['Sector'] == "Energy"]

In [13]:
sec_df

,Symbol,Name,Sector
0,MMM,3M,Industrials
1,AOS,A. O. Smith,Industrials
2,ABT,Abbott Laboratories,Health Care
3,ABBV,AbbVie,Health Care
4,ABMD,Abiomed,Health Care
...,...,...,...
500,YUM,Yum! Brands,Consumer Discretionary
501,ZBRA,Zebra Technologies,Information Technology
502,ZBH,Zimmer Biomet,Health Care
503,ZION,Zions Bancorp,Financials


#### Returns a DF with ROIs for all Stocks

In [14]:
def get_rois_for_stocks(stock_df):
    # Will hold all tickers & stock rois
    tickers = []
    rois = []

    # iterrows provides the index and column for each row in the DF
    for index, row in stock_df.iterrows():
        df = get_df_from_csv(row['Symbol'])
    
        # If we can't find the ticker delete it from the dataframe
        if df is None:
            pass
            # print(row['Symbol'], " is not available")
        else:
            tickers.append(row['Symbol'])
            sdate, edate = get_valid_dates(df, '2018-01-01', '2020-12-31')
            df = df.set_index(['Date'])
            roi = roi_between_dates(df, sdate, edate)
            rois.append(roi)
    return pd.DataFrame({'Ticker':tickers, 'ROI':rois})

In [15]:
industrial = get_rois_for_stocks(indus_df)
health_care = get_rois_for_stocks(health_df)
it = get_rois_for_stocks(it_df)
commun = get_rois_for_stocks(comm_df)
staple = get_rois_for_stocks(staple_df)
discretion = get_rois_for_stocks(discretion_df)
utility = get_rois_for_stocks(utility_df)
finance = get_rois_for_stocks(financial_df)
material = get_rois_for_stocks(material_df)
restate = get_rois_for_stocks(restate_df)
energy = get_rois_for_stocks(energy_df)

In [16]:
industrial

,Ticker,ROI
0,AOS,-0.060680
1,ALK,-0.273133
2,ALLE,0.504415
3,AAL,-0.694661
4,AME,0.698375
5,BA,-0.243300


I only have few stocks, so the results of the following top depends from that issue.

#### Top Industrials

In [17]:
industrial.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
4,AME,0.698375
2,ALLE,0.504415
0,AOS,-0.060680
5,BA,-0.243300
1,ALK,-0.273133


#### Top Healthcare

In [18]:
health_care.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
10,BIO,1.414630
4,ALGN,1.379040
0,ABT,0.958374
3,A,0.795687
2,ABMD,0.684243


#### Top Information Tech

In [19]:
it.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
2,AMD,7.352459
7,AAPL,2.201818
10,ADSK,1.850448
1,ADBE,1.814406
6,ANSS,1.449007


#### Top Communications

In [21]:
commun.sort_values(by=['ROI'], ascending=False).head(20)

,Ticker,ROI
0,ATVI,0.47251


#### Top Staples

In [22]:
staple.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
0,ADM,0.385834
1,CAG,0.047920


#### Top Discretionary

In [23]:
discretion.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
1,AMZN,1.739195
3,AZO,0.609471
4,BBY,0.563621
2,APTV,0.540217
0,AAP,0.500939


#### Top Utilities

In [24]:
utility.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
0,AES,1.402913
3,AWK,0.819223
1,AEE,0.450588
2,AEP,0.271590
4,ATO,0.198211


#### Top Finance

In [25]:
finance.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
6,AJG,1.112315
5,AON,0.659335
9,BLK,0.536003
7,AIZ,0.466653
2,AXP,0.280488


#### Top Materials

In [26]:
material.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
3,BLL,1.474667
0,APD,0.771643
2,AVY,0.421215
1,ALB,0.180334


#### Top Real Estate

In [27]:
restate.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
1,AMT,0.685783
0,ARE,0.492708
2,AVB,0.005870
3,BXP,-0.189743


#### Top Energy

In [28]:
energy.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
0,APA,-0.654885


#### Portfolio List

In [29]:
port_list = ["GNRC", "DXCM", "AMD", "NFLX", "COST", "TGT", "AES", "MSCI", 
             "NEM", "SBAC", "HES"]

#### Check for Correlation

In [ ]:
mult_df = merge_df_by_column_name('daily_return',  '2018-01-01', 
                                  '2020-12-31', *port_list)
mult_df

# Generate a Correlation Matrix
mult_df.corr()

In [ ]:
# Get the number of samples
days = len(mult_df.index)
days

In [ ]:
def get_stock_price_on_date(ticker, date):
    df = get_df_from_csv(ticker)
    df = df.set_index(['Date'])
    return df.loc[date,'Adj Close']

In [ ]:
# 01-04-2021 is the 1st date in 2021
print("GNRC $", get_stock_price_on_date('GNRC', '2021-01-04')) # 2 449.12
print("DXCM $", get_stock_price_on_date('DXCM', '2021-01-04')) # 1 357.74
print("AMD $", get_stock_price_on_date('AMD', '2021-01-04')) # 4 369.2
print("NFLX $", get_stock_price_on_date('NFLX', '2021-01-04')) # 1 522.86
print("COST $", get_stock_price_on_date('COST', '2021-01-04')) # 1 377.88
print("TGT $", get_stock_price_on_date('TGT', '2021-01-04')) # 3 527.46
print("AES $", get_stock_price_on_date('AES', '2021-01-04')) # 18 410.76
print("MSCI $", get_stock_price_on_date('MSCI', '2021-01-04')) # 1 435.40
print("NEM $", get_stock_price_on_date('NEM', '2021-01-04')) # 7 434.49
print("SBAC $", get_stock_price_on_date('SBAC', '2021-01-04')) # 2 548.84
print("HES $", get_stock_price_on_date('HES', '2021-01-04')) # 9 472.23

# Total Investment
tot_inv = 449.12 + 357.74 + 369.2 + 522.86 + 377.88 + 527.46 + 410.76 + \
435.40 + 434.49 + 548.84 + 472.23
tot_inv

#### Returns Portfolio Weight

In [ ]:
def get_port_weight(price, total):
    return price / total

In [ ]:
gnrc_w = get_port_weight(449.12, 4905.98)
dxcm_w = get_port_weight(357.74, 4905.98)
amd_w = get_port_weight(369.2, 4905.98)
nflx_w = get_port_weight(522.86, 4905.98)
cost_w = get_port_weight(377.88, 4905.98)
tgt_w = get_port_weight(527.46, 4905.98)
aes_w = get_port_weight(410.76, 4905.98)
msci_w = get_port_weight(435.40, 4905.98)
nem_w = get_port_weight(434.49, 4905.98)
sbac_w = get_port_weight(548.84, 4905.98)
hes_w = get_port_weight(472.23, 4905.98)

wts = np.array([gnrc_w, dxcm_w, amd_w, nflx_w, cost_w, tgt_w, aes_w, msci_w, \
                nem_w, sbac_w, hes_w])
wts

port_var = np.dot(wts.T, np.dot(mult_df.cov() * 756, wts))
print("Portfolio Covariance :", port_var)
print("GNRC Var :", mult_df["GNRC"].var() * 756)
print("NFLX Var :", mult_df["NFLX"].var() * 756)

#### Different Types of Risk

There is risk that you can limit through diversifaction (Idiosyncratic) and risk that you can't (Systematic). Systematic risk is caused by unforeseen conditions such as wars, recessions, natural disasters, dramatic changes in consumer actions, etc.

Data tells us that if we make a portfolio made up of approximately 25 stocks that aren't correlated that we can dramatically lower idiosyncratic risk. That is one reason why people invest in indexes. You can further lower risk by investing in other countries, bonds and cash.

#### Diversifiable Risk

In [ ]:
# Diversifiable Risk = Portfolio Variance - All Weighted Variances
def calc_diversifiable_risk(df, tickers, weights):
    # Gets number of days
    days = len(mult_df.index)
    # Calculate covariance of portfolio
    port_covar = np.dot(weights.T, np.dot(df.cov() * days, weights)) 
    
    i = 0
    while i < len(tickers):
        wt_sq = weights[i] ** 2
        stk_var = mult_df[tickers[i]].var() * days
        wt_var = wt_sq * stk_var
        port_covar = port_covar - wt_var
        i += 1
    return port_covar

In [ ]:
div_risk = calc_diversifiable_risk(mult_df, port_list, wts)
div_risk

#### Find Risk that Can't be Diversified

In [ ]:
print("Systematic Risk :", (port_var - div_risk))